In [1]:
import io
import json
import requests
import pandas as pd

### Token
enter your token below:

In [2]:
token = 'xxx'

### API wrapper

In [3]:
class RenewablesNinjaAPI:
    def __init__(self, token):
        self.base_url = 'http://renewables.ninja/api/'
        self.session = requests.Session()
        self.token = token
        self.header = {'Authorization': f'Token {token}'}
        self.session.headers.update(self.header)
   
    def get_models(self):
        url = '/'.join([self.base_url, 'models'])
        response = self.session.get(url)
        data = json.loads(response.text)
        return data
    
    def get_wind(self, lat=56, lon=-3, date_from='2014-01-01', date_to='2014-02-28', capacity=1, dataset='merra2', height=100, turbine='Vestas+V80+2000'):
        get_url = (f'data/wind?&lat={lat}&lon={lon}&date_from={date_from}'
                   f'&date_to={date_to}&capacity={capacity}&dataset={dataset}'
                   f'&height={height}&turbine={turbine}&format=csv')
        url = '/'.join([self.base_url, get_url])
        response = self.session.get(url)
        data = pd.read_csv(io.BytesIO(response.content), header=[0,1])
        return data
    
    def get_limit(self):
        url = '/'.join([self.base_url, 'limits'])
        response = self.session.get(url)
        return response.text       
        
    def get_pv(self):
        pass

### Get Data

In [4]:
api = RenewablesNinjaAPI(token)

In [5]:
wind_data = api.get_wind()

In [7]:
wind_data.head()

,time,output
,UTC,kW
0,2014-01-01 00:00,0.533
1,2014-01-01 01:00,0.475
2,2014-01-01 02:00,0.466
3,2014-01-01 03:00,0.459
4,2014-01-01 04:00,0.461


In [8]:
wind_data.shape

(1416, 2)

### Request limits

- api works fine, but requests are limited to 5/day without account
- auth token in header is not recognised and development with only 5 reqs/day is tedious

In [9]:
lim = api.get_limit()
lim

'{"burst":"6/minute","sustained":"5/day"}'

In [19]:
api.session.headers

{'User-Agent': 'python-requests/2.18.4', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Authorization': 'Token xxx_censored'}